## Dash app

In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go 
import plotly.express as px
import pandas as pd 
import numpy as np
import nCoV2019
from dash.dependencies import Input, Output

# 获取数据
nCov = nCoV2019.qq_source()

# fig1 国内新增感染  由 callback 绘制
fig1=go.Figure()

# fig2 全球治愈率比较  由 callback 绘制
fig2=go.Figure()

# fig3 国内每日增长
df = nCov.chinaDayADD
df.date = pd.to_datetime(df.date,format="%m.%d",errors='ignore')
df.date = df.date.apply(lambda dt: dt.replace(year=2020))

fig3=go.Figure(go.Scatter(x=df.date, y=df.heal,mode='lines+markers',name='治愈',marker={"size":8,"color":"green"}))
fig3.add_scatter(x=df.date, y=df.dead,mode='lines+markers',name='死亡',marker={"size":8,"color":"gray"})
fig3.add_scatter(x=df.date, y=df.confirm,mode='lines+markers',name='确诊',marker={"size":8,"color":"blue"})
fig3.add_scatter(x=df.date, y=df.suspect,mode='lines+markers',name='疑似',marker={"size":8,"color":"red"})
fig3.update_layout(template="none",yaxis_title='人数',xaxis_title='日期',title='国内每日新增确诊、疑似、治愈、死亡曲线')

# fig4 国内累计数据
df = nCov.chinaDayList
df.date = pd.to_datetime(df.date,format="%m.%d",errors='ignore')
df.date = df.date.apply(lambda dt: dt.replace(year=2020))
fig4=go.Figure(go.Scatter(x=df.date, y=df.heal,mode='lines+markers',name='治愈',marker={"size":8,"color":"green"}))
fig4.add_scatter(x=df.date, y=df.dead,mode='lines+markers',name='死亡',marker={"size":8,"color":"gray"})
fig4.add_scatter(x=df.date, y=df.confirm,mode='lines+markers',name='确诊',marker={"size":8,"color":"red"})
fig4.add_scatter(x=df.date, y=df.suspect,mode='lines+markers',name='疑似',marker={"size":8,"color":"blue"})
fig4.update_layout(template="none",yaxis_title='人数',xaxis_title='日期',title='累计治愈、死亡曲线')

# fig5 国外疫情爆发情况
df = nCov.global_area
df = df[-df['country'].isin(['中国'])]
fig5 = go.Figure(data=[go.Pie(labels=df.country, values=df.total_confirm, hole=.5)])
fig5.update_layout(title='国外疫情情况')

# fig6 国内疫情爆发情况
df = nCov.china
fig6 = go.Figure(data=[go.Pie(labels=df.city, values=df.total_confirm, hole=.5)])
fig6.update_layout(title='国内疫情情况')

# fig7 各地单日新增率 由 callback 绘制
fig7=go.Figure()
# fig8 geo 可视化，经纬度在 china_geo.csv
df = nCov.china
a=pd.read_csv('china_geo.csv')
a=pd.merge(df,a,on='city')
# 湖北确诊数量实在太多了，log2转换后立方，减少不同地区的悬殊，同时能分别大小。
_range2 = [round(np.log2(i)*3) for i in a.total_confirm ]

fig8 = go.Figure(go.Scattermapbox(
        lat=a.latitude,
        lon=a.longitude,
        mode='markers',
    marker={"size":_range2,"color":_range2,"colorscale": "YlOrRd"},
    hovertemplate ='<i>%{text}</i>'+'<br>%{customdata}',text=a.city,customdata=a.total_confirm))

fig8.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken='pk.eyJ1IjoidGVzdDc4NSIsImEiOiJjazZnYWxseHIxd2x4M2VwczJueTB5cmdnIn0.cWsGdteJC5QbdLPOSk5cLA',
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=30,
            lon=100
        ),
        pitch=0,
        zoom=2
    ))

# 概览
a = nCov.get('中国').total_confirm[0]
b = nCov.get('中国').total_suspect[0]
c = nCov.get('中国').total_heal[0]
d = nCov.get('中国').total_dead[0]
# CSS 采用 float + min-width 简单响应布局
style={
    "boder":{
    "border-radius": "15px",
    "background-color": "#f9f9f9",
    "margin": "10px",
    "padding": "15px",
    "position": "relative",
    "box-shadow": "2px 2px 2px gray"},
    "float":{
    "position": "relative",
    "float":"left",
    "border-radius": "15px",
    "background-color": "#f9f9f9",
    "margin": "10px",
    "padding": "15px",
    "box-shadow": "2px 2px 2px gray",
    "width":"45%",
    "min-width":"400px"},
    "left_top":{
    "width":"45%"},
}

app = dash.Dash()
avail = ['中国']+[i for i in nCov.china.city] # 获取国内全部省份
app.layout = html.Div([html.Div([html.Center([html.H1('2019nCoV Dash'),""]),
              html.Center([html.H2(f'确诊 {a} | 疑似 {b} | 治愈 {c} | 死亡 {d} ',style=style['boder'])])]),
    html.Div(['新增情况',
    dcc.Dropdown(
        options=[{'label':i,'value':i} for i in avail],id="select",
        value='湖北',style=style['left_top']
    ),
    '死亡率/治愈率',
    dcc.Dropdown(
        options=[{'label':i,'value':i} for i in ['中国','全球']],id="select2",
        value='全球',style=style['left_top']
    ),html.Hr(),
    dcc.RadioItems(
        id='reflesh',
        options=[{'label': k, 'value': k} for k in ['更新数据(需手动刷新)','使用缓存数据']],
        value='使用缓存数据'
    )],style=style['float']),
    html.Div([
        dcc.Graph(
        figure = fig1
    )],id='fig1',style=style['float']),
    html.Div([dcc.Graph(
            figure = fig2
    )],id='fig2',style=style['float']),
    html.Div([dcc.Graph(
            id='fig3',
            figure = fig3
    )],style=style['float']),
    html.Div([dcc.Graph(
            id='fig4',
            figure = fig4
    )],style=style['float']),
    html.Div([dcc.Graph(
            id='fig5',
            figure = fig5
    )],style=style['float']),
    html.Div([dcc.Graph(
            id='fig6',
            figure = fig6
    )],style=style['float']),
    html.Div([dcc.Graph(
            figure = fig7
    )],id='fig7',style=style['float']),
    html.Div([dcc.Graph(
            id='fig8',
            figure = fig8
    )],style=style['float'])  
])


## callback
@app.callback(
    Output(component_id='fig7', component_property='children'),
    [Input(component_id='reflesh', component_property='value')]
)
def update_fig7(input_value,fig7=fig7):
    if input_value == '更新数据(需手动刷新)':
        global nCov
        nCov = nCoV2019.qq_source()
    df = nCov.china
    df['rate'] = round(df.confirm/df.total_confirm,2)
    df = df.sort_values(by=['rate','confirm'],ascending=False,inplace = False)
    fig7 = px.bar(df, x='city', y='rate',
                 hover_data=['confirm'],hover_name=df.city, color=df.confirm,text=df.rate,
           color_continuous_scale=px.colors.sequential.thermal,
                  template="none")
    fig7.update_layout(title='各地单日确诊增长率',xaxis_title = '地区',yaxis_title='确诊增长率')
    return dcc.Graph(
        figure = fig7)

@app.callback(
    Output(component_id='fig1', component_property='children'),
    [Input(component_id='select', component_property='value')]
)
def update_fig1(input_value):
    lc=input_value
    if lc == '中国':
        df = nCov.china
    else:
        df = nCov.get(lc)[1:]
    df = df.sort_values(by=['total_confirm'],ascending=False,)
    fig1 = px.bar(df,x=df.city,y=df.total_confirm,color=df.confirm,text=df.confirm,
           color_continuous_scale=px.colors.sequential.YlOrRd,
          title=f'{lc} 新增感染人数: {sum(df.confirm)}',
           template="none")
    fig1.update_layout(xaxis_title='城市',yaxis_title='确诊总数')
    return dcc.Graph(
        figure = fig1
    )
@app.callback(
    Output(component_id='fig2', component_property='children'),
    [Input(component_id='select2', component_property='value')]
)
def update_fig2(input_value):
    if input_value == '中国':
        df = nCov.china
        x = df.city
    else:
        df = nCov.global_area
        x = df.country

    fig2 = go.Figure(data=[
        go.Bar(name='治愈率', x=x, y=df.total_healRate,marker={"color":'green'},
            hovertemplate =
            '<i>治愈率</i>: %{y:.2f}%'+'<br>确诊:%{text}'+'<br>治愈:%{customdata}',
              text=df.total_confirm,
              customdata=df.total_heal),
        go.Bar(name='死亡率', x=x, y=df.total_deadRate,marker={"color":'red'},
            hovertemplate =
            '<i>死亡率</i>: %{y:.2f}%'+'<br>确诊:%{text}'+'<br>死亡:%{customdata}',
              text=df.total_confirm,
              customdata=df.total_dead)
    ])
    fig2.update_layout(template="none",yaxis_title='',xaxis_title='受感染国家/城市（按感染人数排序）',title='各地区死亡率、治愈率')

    return dcc.Graph(
        figure = fig2
    )

app.run_server()

截止 2020-02-11 15:31:41; 2019nCoV 已蔓延 25 个国家/地区
中国累计 42718 例确诊，自昨日00:00新增2483
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Feb/2020 16:06:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2020 16:06:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2020 16:06:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2020 16:06:49] "GET /_favicon.ico?v=1.8.0 HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2020 16:06:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2020 16:06:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2020 16:06:50] "POST /_dash-update-component HTTP/1.1" 200 -
